In [1]:
import json
import requests

In [2]:
import geopandas as gpd
import pandas as pd

## Open File

In [3]:
file_path = "data/maakond_shp/maakond.shp"
estonia_counties_gdf = gpd.read_file(file_path)

## Get Open Population data

In [4]:
body = {
    "query": [
        {"code": "Aasta", "selection": {"filter": "item", "values": ["2018", "2025"]}},
        {"code": "Sugu", "selection": {"filter": "item", "values": ["1"]}},
        {
            "code": "Maakond",
            "selection": {
                "filter": "item",
                "values": [
                    "00",
                    "37",
                    "784",
                    "39",
                    "45",
                    "50",
                    "52",
                    "56",
                    "60",
                    "64",
                    "68",
                    "71",
                    "74",
                    "79",
                    "793",
                    "81",
                    "84",
                    "87",
                ],
            },
        },
        {"code": "Rahvus", "selection": {"filter": "item", "values": ["5"]}},
    ],
    "response": {"format": "json-stat2"},
}
r = requests.post(
    "https://andmed.stat.ee/api/v1/en/stat/RV0222U", data=json.dumps(body)
)

In [5]:
data = r.json()
values = data["value"]
formatted_values = [
    {"2018": values[i], "2025": values[i + 1]} for i in range(0, len(values), 2)
]

In [6]:
formatted_dict = {
    "MKOOD": [
        key.zfill(4) for key in data["dimension"]["Maakond"]["category"]["index"].keys()
    ],
    "2025": values[int(len(values) / 2) :],
    "2018": values[: int(len(values) / 2)],
}

In [7]:
population_df = pd.DataFrame(formatted_dict)

In [9]:
population_df["DIFF"] = population_df["2025"] - population_df["2018"]
population_df["PERCENT"] = (
    (population_df["DIFF"] / population_df["2018"]) * 100
).round(decimals=0)

In [10]:
population_df[population_df["MKOOD"] == "0000"]

,MKOOD,2025,2018,DIFF,PERCENT
0,0000,73597,23310,50287,216.0


## Merge data and export

In [15]:
estonian_pop_gdf = estonia_counties_gdf.merge(population_df, on="MKOOD", how="left")

In [16]:
estonian_pop_gdf = estonian_pop_gdf.to_crs(3300)
estonian_pop_gdf.to_file("data/Population_Change_Ukrainians_In_Estonia_2018_2025.gpkg")

In [17]:
estonian_pop_gdf["2018"].sum()

np.int64(23287)

In [18]:
estonian_pop_gdf["2025"].sum()

np.int64(73597)